In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer
pd.set_option('display.max_columns', None)
# eye candy plots
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-light.mplstyle')

In [2]:
df = pd.read_csv("../../data/processed/3_aggregated_pueblos.csv")
df.shape
df = df.dropna(subset=['koppen_climate'])

In [3]:
df = df.drop(['cmun', 'municipality', 'towns_in_vicinity', 'town_size', 'connectivity_category', 'description', 'closest_distance_airport', 'nearest_airport','nearest_train_station', 'closest_distance_train', 'closest_distance_hospital', 'nearest_hospital', 'closest_distance_school', 'nearest_school', 'longitude', 'latitude', 'male', 'total_population'], axis=1)

In [4]:
df

,province,altitude,0-17,18-24,25-34,35-54,55+,female,koppen_climate,number_of_houses,connectivity_score,economy_score,economy_score_area,autonomous_community,hospital_distance_score,school_distance_score,hospital_score_area,school_score_area,train_distance_score,airport_distance_score,transport_score
0,Bizkaia,18.95764,302,96,101,463,584,736,Cfb,784.0,0.7180,4.10,16.823071,País Vasco,6,10,3.0,5.0,1,6,3.5
1,A Coruña,151.44100,265,119,160,648,1199,1218,Csb,1277.0,0.8460,5.90,14.034524,Galicia,4,10,2.0,5.0,10,6,8.0
2,A Coruña,362.87660,119,61,95,364,694,667,Cfb,788.0,0.6420,0.00,8.933750,Galicia,3,10,1.5,5.0,6,6,6.0
3,Bizkaia,107.88860,58,28,32,113,151,181,Cfb,213.0,0.7410,0.00,15.153360,País Vasco,4,10,2.0,5.0,1,6,3.5
4,Bizkaia,44.55854,528,226,291,1016,1251,1662,Cfb,1497.0,0.9425,35.25,31.005961,País Vasco,10,10,5.0,5.0,1,10,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7611,Cádiz,15.97273,3720,1797,2572,6793,7843,11399,Csa,11870.0,0.9915,69.35,98.668750,Andalucía,2,10,1.0,5.0,1,4,2.5
7612,Cádiz,10.40490,11924,4933,8008,19871,20251,32951,Csa,29253.0,0.9830,167.90,160.098442,Andalucía,10,10,5.0,5.0,8,10,9.0
7613,Cádiz,22.40544,24311,10234,13808,38778,37916,63493,Csa,54701.0,0.9525,383.25,248.112500,Andalucía,10,10,5.0,5.0,10,10,10.0
7614,Cádiz,14.66109,2983,1261,2021,6330,6062,9332,Csa,12100.0,0.9335,92.60,134.566667,Andalucía,4,10,2.0,5.0,4,6,5.0


In [5]:
ohe = OneHotEncoder(drop='first', sparse_output=False)
scaler = StandardScaler()

pipeline = Pipeline(steps=[
    ('ohe', OneHotEncoder(drop='first', sparse_output=False))
])

pipeline.fit(df[['koppen_climate', 'autonomous_community', 'province']])

Pipeline(steps=[('ohe', OneHotEncoder(drop='first', sparse_output=False))])

In [6]:
fe_transformer = ColumnTransformer(transformers=[
    ('transf_cat', pipeline, ['koppen_climate', 'autonomous_community', 'province']),
    ('scaled', scaler, [
        'altitude', '0-17', '18-24', '25-34', '35-54', '55+',
        'female', 'number_of_houses', 'connectivity_score',
        'economy_score', 'economy_score_area',
        'hospital_distance_score', 'school_distance_score', 'hospital_score_area',
        'school_score_area', 'train_distance_score', 'airport_distance_score', 'transport_score'
    ])
], remainder='drop')

fe_transformer.fit(df)

ColumnTransformer(transformers=[('transf_cat',
                                 Pipeline(steps=[('ohe',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False))]),
                                 ['koppen_climate', 'autonomous_community',
                                  'province']),
                                ('scaled', StandardScaler(),
                                 ['altitude', '0-17', '18-24', '25-34', '35-54',
                                  '55+', 'female', 'number_of_houses',
                                  'connectivity_score', 'economy_score',
                                  'economy_score_area',
                                  'hospital_distance_score',
                                  'school_distance_score',
                                  'hospital_score_area', 'school_score_area',
                                  'train_distance_score',
                                  'airport_distance_score',
                                  'transport_score'])])

In [10]:
df_train = fe_transformer.transform(df)


array([[ 0.        ,  0.        ,  1.        , ..., -1.06392277,
         1.15594514, -0.28383358],
       [ 0.        ,  0.        ,  0.        , ...,  1.79474141,
         1.15594514,  1.95827505],
       [ 0.        ,  0.        ,  1.        , ...,  0.52422399,
         1.15594514,  0.96178232],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  1.79474141,
         3.24792356,  2.95476777],
       [ 0.        ,  0.        ,  0.        , ..., -0.11103471,
         1.15594514,  0.46353596],
       [ 0.        ,  0.        ,  0.        , ..., -0.74629342,
         3.24792356,  0.96178232]])

In [13]:
pd.DataFrame(df_train).to_csv("../../data/processed/cluster_model_train.csv", index=False)